In this notebook we are testing the zero-shot approach for classification called binoculars with falcon-7b and falcon-7b-instruct. The dataset contains various texts from different domains such as Arxiv, Wikipedia, Wikihow and Reddit. Furthermore, the texts are written by human and also rewritten by the LLMs 'bloomz', 'chatgpt', 'cohere', 'davinci'.

In [ ]:
%pip install gdown
#upgrade
%pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 5.1.0
    Uninstalling gdown-5.1.0:
      Successfully uninstalled gdown-5.1.0


In [ ]:
%pip install accelerate bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import os
import torch.nn.functional as F
import transformers
import gdown

from tqdm import tqdm
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler
from timeit import default_timer as timer
from os import walk
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_curve, auc, brier_score_loss
from typing import Union
from google.colab import files

## Constants

In [ ]:
TEST_SET_FRACTION = 0.3
BATCH_SIZE = 8

In [ ]:
ROOT_DATA = './'
M4_DATA_FOLDER_PATH = f'{ROOT_DATA}m4-unified'

In [ ]:
LLMS = ['bloomz', 'chatgpt', 'cohere', 'davinci', 'human']
DOMAINS = ['arxiv', 'wikihow', 'reddit', 'wikipedia']

In [27]:
LLMS_TO_PROCESS = ['human']
DOMAINS_TO_PROCESS = ['reddit']

In [ ]:
OBSERVER = "mistralai/Mistral-7B-v0.1"
PERFORMER = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
# selected using Falcon-7B and Falcon-7B-Instruct at bfloat16
BINOCULARS_ACCURACY_THRESHOLD = 0.9015310749276843  # optimized for f1-score
BINOCULARS_FPR_THRESHOLD = 0.8536432310785527  # optimized for low-fpr [chosen at 0.01%]

DEVICE_1 = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE_2 = "cuda:1" if torch.cuda.device_count() > 1 else DEVICE_1

## Metrics

In [ ]:
ce_loss_fn = torch.nn.CrossEntropyLoss(reduction="none")
softmax_fn = torch.nn.Softmax(dim=-1)

def perplexity(encoding: transformers.BatchEncoding,
               logits: torch.Tensor,
               median: bool = False,
               temperature: float = 1.0):


    shifted_logits = logits[..., :-1, :].contiguous() / temperature
    shifted_labels = encoding.input_ids[..., 1:].contiguous()
    shifted_attention_mask = encoding.attention_mask[..., 1:].contiguous()

    if median:
        #shifter_logits.transpose(1, 2).shape: [batch_size, vocab_size, max_text_length - 1] (-1 is because we remove the logits for the last token)
        ce_nan = (ce_loss_fn(shifted_logits.transpose(1, 2), shifted_labels).
                  masked_fill(~shifted_attention_mask.bool(), float("nan")))
        ppl = np.nanmedian(ce_nan.cpu().float().numpy(), 1)

    else:
        loss = ce_loss_fn(shifted_logits.transpose(1, 2), shifted_labels)
        ppl = (loss *
               shifted_attention_mask).sum(1) / shifted_attention_mask.sum(1)
        ppl = ppl.to("cpu").float().numpy()

    return ppl


def entropy(p_logits: torch.Tensor,
            q_logits: torch.Tensor,
            encoding: transformers.BatchEncoding,
            pad_token_id: int,
            median: bool = False,
            sample_p: bool = False,
            temperature: float = 1.0):
    vocab_size = p_logits.shape[-1]
    total_tokens_available = q_logits.shape[-2]
    p_scores, q_scores = p_logits / temperature, q_logits / temperature

    p_proba = softmax_fn(p_scores).view(-1, vocab_size)

    if sample_p:
        p_proba = torch.multinomial(p_proba.view(-1, vocab_size), replacement=True, num_samples=1).view(-1)

    q_scores = q_scores.view(-1, vocab_size)

    ce = ce_loss_fn(input=q_scores, target=p_proba).view(-1, total_tokens_available)
    padding_mask = (encoding.input_ids != pad_token_id).type(torch.uint8)

    if median:
        ce_nan = ce.masked_fill(~padding_mask.bool(), float("nan"))
        agg_ce = np.nanmedian(ce_nan.cpu().float().numpy(), 1)
    else:
        agg_ce = (((ce * padding_mask).sum(1) / padding_mask.sum(1)).to("cpu").float().numpy())

    return agg_ce

In [ ]:
def assert_tokenizer_consistency(model_id_1, model_id_2):
    identical_tokenizers = (
            AutoTokenizer.from_pretrained(model_id_1).vocab
            == AutoTokenizer.from_pretrained(model_id_2).vocab
    )
    if not identical_tokenizers:
        raise ValueError(f"Tokenizers are not identical for {model_id_1} and {model_id_2}.")

## Quantization config

## Models

In [ ]:
torch.set_grad_enabled(False)

huggingface_config = {
    # Only required for private models from Huggingface (e.g. LLaMA models)
    "TOKEN": os.environ.get("HF_TOKEN", None)
}

class Binoculars(object):
    def __init__(self,
                 observer_name_or_path: str = OBSERVER,
                 performer_name_or_path: str = PERFORMER,
                 use_bfloat16: bool = True,
                 max_token_observed: int = 512,
                 mode: str = "low-fpr",
                 ) -> None:
        assert_tokenizer_consistency(observer_name_or_path, performer_name_or_path)

        self.change_mode(mode)
        self.observer_model = AutoModelForCausalLM.from_pretrained(observer_name_or_path,
                                                                   device_map={"": DEVICE_1},
                                                                   trust_remote_code=True,
                                                                   token=huggingface_config["TOKEN"],
                                                                   load_in_4bit=True
                                                                   )
        self.performer_model = AutoModelForCausalLM.from_pretrained(performer_name_or_path,
                                                                    device_map={"": DEVICE_2},
                                                                    trust_remote_code=True,
                                                                    load_in_4bit=True,
                                                                    token=huggingface_config["TOKEN"]
                                                                    )
        self.observer_model.eval()
        self.performer_model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(observer_name_or_path)
        if not self.tokenizer.pad_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.max_token_observed = max_token_observed

    def change_mode(self, mode: str) -> None:
        if mode == "low-fpr":
            self.threshold = BINOCULARS_FPR_THRESHOLD
        elif mode == "accuracy":
            self.threshold = BINOCULARS_ACCURACY_THRESHOLD
        else:
            raise ValueError(f"Invalid mode: {mode}")

    def _tokenize(self, batch: list[str]) -> transformers.BatchEncoding:
        batch_size = len(batch)
        encodings = self.tokenizer(
            batch,
            return_tensors="pt",
            padding="longest" if batch_size > 1 else False,
            truncation=True,
            max_length=self.max_token_observed,
            return_token_type_ids=False).to(self.observer_model.device)
        return encodings

    @torch.inference_mode()
    def _get_logits(self, encodings: transformers.BatchEncoding) -> torch.Tensor:
        observer_logits = self.observer_model(**encodings.to(DEVICE_1)).logits
        performer_logits = self.performer_model(**encodings.to(DEVICE_2)).logits
        if DEVICE_1 != "cpu":
            torch.cuda.synchronize()
        return observer_logits, performer_logits

    def compute_score(self, input_text: Union[list[str], str]) -> Union[float, list[float]]:
        batch = [input_text] if isinstance(input_text, str) else input_text
        encodings = self._tokenize(batch)
        observer_logits, performer_logits = self._get_logits(encodings)
        ppl = perplexity(encodings, performer_logits)
        x_ppl = entropy(observer_logits.to(DEVICE_1), performer_logits.to(DEVICE_1),
                        encodings.to(DEVICE_1), self.tokenizer.pad_token_id)
        binoculars_scores = ppl / x_ppl
        binoculars_scores = binoculars_scores.tolist()
        return binoculars_scores[0] if isinstance(input_text, str) else binoculars_scores

    def predict(self, input_text: Union[list[str], str]) -> Union[list[str], str]:
        binoculars_scores = np.array(self.compute_score(input_text))

        return binoculars_scores


## Data

#### Using hosted environment

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !unzip /content/gdrive/MyDrive/m4-unified.zip

#### Using custom GCE VM

In [ ]:
url = "https://drive.google.com/file/d/1Ky57-fv5Bz7fVVTIT3vdTvTQd8mPCePm/view?usp=drive_link"
output = "m4-unified.zip"
gdown.download(url, output, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1Ky57-fv5Bz7fVVTIT3vdTvTQd8mPCePm
From (redirected): https://drive.google.com/uc?id=1Ky57-fv5Bz7fVVTIT3vdTvTQd8mPCePm&confirm=t&uuid=780a80e1-5996-4411-8b33-c104c6278834
To: /content/m4-unified.zip
100%|██████████| 49.9M/49.9M [00:03<00:00, 14.4MB/s]


'm4-unified.zip'

In [ ]:
!unzip m4-unified.zip

Archive:  m4-unified.zip
   creating: m4-unified/arxiv/
  inflating: m4-unified/arxiv/arxiv_bloomz.jsonl  
  inflating: m4-unified/arxiv/arxiv_chatGPT.jsonl  
  inflating: m4-unified/arxiv/arxiv_cohere.jsonl  
  inflating: m4-unified/arxiv/arxiv_davinci.jsonl  
  inflating: m4-unified/arxiv/arxiv_flant5.jsonl  
  inflating: m4-unified/arxiv/arxiv_human.jsonl  
   creating: m4-unified/reddit/
  inflating: m4-unified/reddit/reddit_bloomz.jsonl  
  inflating: m4-unified/reddit/reddit_chatGPT.jsonl  
  inflating: m4-unified/reddit/reddit_cohere.jsonl  
  inflating: m4-unified/reddit/reddit_davinci.jsonl  
  inflating: m4-unified/reddit/reddit_dolly.jsonl  
  inflating: m4-unified/reddit/reddit_flant5.jsonl  
  inflating: m4-unified/reddit/reddit_human.jsonl  
   creating: m4-unified/wikihow/
  inflating: m4-unified/wikihow/wikihow_bloomz.jsonl  
  inflating: m4-unified/wikihow/wikihow_chatGPT.jsonl  
  inflating: m4-unified/wikihow/wikihow_cohere.jsonl  
  inflating: m4-unified/wikihow/wik

## Test

In [ ]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size

    def __getitem__(self, index):
        return (
            self.X[index],
            self.y[index]
        )

In [ ]:
model = Binoculars()
tokenizer = model.tokenizer

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def run_detector(model, input_str_list: list[str]):
    return model.predict(input_str_list)

In [28]:
dir_path, dir_names, file_names = next(walk(M4_DATA_FOLDER_PATH))

scores_file_names = []

for dir in DOMAINS_TO_PROCESS:
    dataset_folder_path, _, dataset_names = next(walk(os.path.join(dir_path, dir)))

    for dataset_name in dataset_names:
        dataset_llm = Path(dataset_name).stem.split('_')[1].lower()
        if dataset_llm not in LLMS_TO_PROCESS:
          continue
        temp_df = pd.read_json(path_or_buf=f'{dataset_folder_path}/{dataset_name}', lines=True)
        temp_df['domain'] = dir
        temp_df['dataset_name'] = Path(dataset_name).stem
        temp_df['is_llm'] = 0 if 'human' in dataset_name else 1
        print(dataset_name, 0 if 'human' in dataset_name else 1)

        display(temp_df)

        dataset = TextDataset(temp_df['text'], temp_df['is_llm'])
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, drop_last=False)

        all_scores = []
        for batch in tqdm(dataloader):
          scores = run_detector(model, list(batch[0]))
          all_scores.extend(scores)

        scores_file_name = f'{dataset_folder_path}/{dataset_name}_scores.csv'
        scores_file_names.append(scores_file_name)
        pd.DataFrame(all_scores, columns=["scores"]).to_csv(scores_file_name)
        files.download(scores_file_name)

reddit_human.jsonl 0


,text,domain,dataset_name,is_llm
0,Henry died in a joust against the captain of h...,reddit,reddit_human,0
1,"In 1801, James Monroe and Robert R. Livingston...",reddit,reddit_human,0
2,Good question! [I answered this a few years ba...,reddit,reddit_human,0
3,Watergate is an incredibly interesting period ...,reddit,reddit_human,0
4,"No, in medieval Europe, there were no restric...",reddit,reddit_human,0
...,...,...,...,...
2995,Excellent question. A bunch of people want to ...,reddit,reddit_human,0
2996,"Chess is slightly biased toward white, given e...",reddit,reddit_human,0
2997,The main factor in speed zoning is supposed to...,reddit,reddit_human,0
2998,I teach a class in sleep physiology and dreami...,reddit,reddit_human,0


100%|██████████| 375/375 [3:11:31<00:00, 30.65s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create a JSON Encoder class
class json_serialize(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
